In [1]:
import pandas as pd
import great_expectations as ge

In [2]:
sample_data_path = ".\historical_data_2022Q4.txt"
column_names = [
        "Credit Score",
        "First Payment Date",
        "First Time Homebuyer Flag",
        "Maturity Date",
        "Metropolitan Statistical Area (MSA) Or Metropolitan Division",
        "Mortgage Insurance Percentage (MI %)",
        "Number of Units",
        "Occupancy Status",
        "Original Combined Loan-to-Value (CLTV)",
        "Original Debt-to-Income (DTI) Ratio",
        "Original UPB",
        "Original Loan-to-Value (LTV)",
        "Original Interest Rate",
        "Channel",
        "Prepayment Penalty Mortgage (PPM) Flag",
        "Amortization Type (Formerly Product Type)",
        "Property State",
        "Property Type",
        "Postal Code",
        "Loan Sequence Number",
        "Loan Purpose",
        "Original Loan Term",
        "Number of Borrowers",
        "Seller Name",
        "Servicer Name",
        "Super Conforming Flag",
        "Pre-HARP Loan Sequence Number",
        "Program Indicator",
        "HARP Indicator",
        "Property Valuation Method",
        "Interest Only (I/O) Indicator",
        "Mortgage Insurance Cancellation Indicator",
    ]
df = pd.read_csv(sample_data_path, delimiter="|", header=None, names=column_names)
context = ge.data_context.DataContext()


In [4]:
context

{
  "anonymous_usage_statistics": {
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "data_context_id": "cedddd2f-b316-4391-8348-e7836cc762f3",
    "explicit_url": false,
    "enabled": true,
    "explicit_id": true
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "my_pandas_datasource": {
      "type": "pandas",
 

In [5]:
suite_name = "freddie_mac_expectation_suite"
context.create_expectation_suite(suite_name, overwrite_existing=True)

{
  "expectation_suite_name": "freddie_mac_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.17.19"
  }
}

In [6]:
ge_df

NameError: name 'ge_df' is not defined

In [7]:
datasource = context.sources.add_pandas(name="my_pandas_datasource1")

In [8]:
name = "fm_dataframe"

In [9]:
data_asset = datasource.add_dataframe_asset(name=name)

In [10]:
my_batch_request = data_asset.build_batch_request(dataframe=df)

In [11]:
suite = context.get_expectation_suite(suite_name)

In [12]:
validator = context.get_validator(batch_request=my_batch_request, expectation_suite=suite)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Mortgage Insurance Cancellation Indicator
0,721,202212,Y,205211,NaN,35,1,P,97,37,...,1,"WELLS FARGO BANK, N.A.","WELLS FARGO BANK, N.A.",NaN,NaN,9,NaN,2,N,N
1,716,202212,N,203711,NaN,0,1,P,68,40,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
2,789,202301,N,205212,NaN,30,1,P,95,28,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,N
3,763,202211,N,203710,NaN,0,1,P,67,39,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
4,792,202212,N,205211,47260.0,0,1,P,76,37,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


In [16]:
validator.expect_column_values_to_be_in_set("Occupancy Status", ["P", "I", "S", "9"])
validator.expect_column_values_to_be_in_set("First Time Homebuyer Flag", ["Y", "N", "9"])
validator.expect_column_value_lengths_to_equal("Occupancy Status", 1)
validator.expect_column_value_lengths_to_equal("First Time Homebuyer Flag", 1)
# Ensure the column values are of type integer
validator.expect_column_values_to_be_of_type("Maturity Date", "int")

# Check that the first two digits are "20"
validator.expect_column_values_to_match_regex("Maturity Date", "^20[0-9]{4}$")
# Validate that the length of each value is exactly 6
validator.expect_column_value_lengths_to_equal("Maturity Date", 6)
# Ensure the column values are of type integer
validator.expect_column_values_to_be_of_type("Credit Score", "int")
# Check the range of the values to be between 300 and 850 or equal to 9999
validator.expect_column_values_to_be_in_set("Credit Score", [9999] + list(range(300, 851)))
# Validate that the length of each value is at most 4
validator.expect_column_value_lengths_to_be_between("Credit Score", min_value=3, max_value=4)

# Check that the length of the string is exactly 2
validator.expect_column_value_lengths_to_equal("Property State", value=2)




Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

MetricResolutionError: Unrecognized numpy/python type: string